In [6]:
# Import necessary libraries
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
with open('pc_usr_data.json', 'r') as file:
    data = json.load(file)

# Explore the structure of the dataset
print("Dataset sample:")
print(json.dumps(data[0], indent=4))  # Print the first item in the dataset

# Extract data into a DataFrame for easier analysis
records = []
for item in data:
    context = item["context"]
    fact = item["fact"]
    for response in item["responses"]:
        record = {
            "context": context,
            "fact": fact,
            "response": response["response"],
            "model": response["model"],
            "understandable_avg": sum(response["Understandable"]) / len(response["Understandable"]),
            "natural_avg": sum(response["Natural"]) / len(response["Natural"]),
            "maintains_context_avg": sum(response["Maintains Context"]) / len(response["Maintains Context"]),
            "engaging_avg": sum(response["Engaging"]) / len(response["Engaging"]),
            "uses_knowledge_avg": sum(response["Uses Knowledge"]) / len(response["Uses Knowledge"]),
            "overall_avg": sum(response["Overall"]) / len(response["Overall"]),
        }
        records.append(record)

df = pd.DataFrame(records)

# Display the first few rows of the DataFrame
print("\nDataFrame preview:")
print(df.head())

# # Summary statistics of the scores
# print("\nScore statistics:")
# print(df.describe())

# # Visualize the distribution of overall scores
# plt.figure(figsize=(10, 6))
# sns.histplot(df["overall_avg"], kde=True, bins=10, color='skyblue')
# plt.title("Distribution of Overall Scores")
# plt.xlabel("Overall Score")
# plt.ylabel("Frequency")
# plt.show()



Dataset sample:
{
    "context": "hi there how are you doing this evening ?\nhi , sitting here with my three dogs watching the olympics !\nnice i do not want to go back to work i am a waitress\ni love being in a polyamorous open relationship !\niol well i wish i was brave enough to do that\nmy father was a salesman , helps my dog walking business now\nthat is nice i've a motorbike don't know what car to get for winter\nvery very cool . sounds fun\nyes i had them put red with blue stripes to be shinny for when racing\nso is my dog , wow so cool\nso what do you do in your spare time ?\nlead singer for a band , music teacher\nwow nice are you really good ?\nmillions of plays on soundcloud\nreally would you share or are you shy\n",
    "fact": "your persona: i also have a dog walking business.\nyour persona: i've three dogs.\nyour persona: my father was a door to door salesman.\nyour persona: i am in an open polyamorous relationship.\nyour persona: i like to watch the olympics.\n",
    "an